In [1]:
import tensorflow_hub as hub
import tensorflow_text as text
import tensorflow as tf

2023-07-03 15:09:28.184726: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-03 15:09:28.306780: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-07-03 15:09:28.306795: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-07-03 15:09:29.019170: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

In [2]:
preprocess_url = 'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3'
encoder_url = 'https://tfhub.dev/tensorflow/bert_en_uncased_L-24_H-1024_A-16/4'

In [3]:
bert_preprocess_model = hub.KerasLayer(preprocess_url)

2023-07-03 15:09:36.785285: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-07-03 15:09:36.785628: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-07-03 15:09:36.785695: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2023-07-03 15:09:36.785750: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2023-07-03 15:09:36.785806: W tensorflow/c

In [4]:
text_test = ['nice movie indeed','I hate python programming']
text_preprocessed = bert_preprocess_model(text_test)
text_preprocessed.keys()

dict_keys(['input_type_ids', 'input_mask', 'input_word_ids'])

In [5]:
bert_model = hub.KerasLayer(encoder_url)

bert_results = bert_model(text_preprocessed)
bert_results.keys()

dict_keys(['pooled_output', 'default', 'encoder_outputs', 'sequence_output'])

In [6]:
import pandas as pd
df = pd.read_csv("tweet.csv")
df.head(5)

,Name,Tweet description
0,Barack Obama,We're seeing the fastest rise in sea-levels in...
1,Barack Obama,RT @WhiteHouse: Watch @POTUS and @FLOTUS share...
2,Barack Obama,See what editorial boards are saying about Sen...
3,Barack Obama,President Obama is at the @CopernicusCtr in Ch...
4,Barack Obama,LIVE: President Obama is answering questions f...


In [7]:
df.groupby('Name').describe()

Tweet description         \
                                count unique   
Name                                           
Barack Obama                     2585   2585   
Cristiano Ronaldo                2032   2031   
Ellen DeGeneres                  2457   2457   
Elon Musk                        2232   2231   
KATY PERRY                       2326   2325   
Kim Kardashian West              2149   2148   
Neil deGrasse Tyson              2525   2515   
Sebastian Ruder                  2374   2374   

                                                                             
                                                                   top freq  
Name                                                                         
Barack Obama         We're seeing the fastest rise in sea-levels in...    1  
Cristiano Ronaldo    The most retweeted will be followed by me and ...    2  
Ellen DeGeneres      Today I’m talking about a topic that affects a...    1  
Elon Musk                                         @tesla_truth Exactly    2  
KATY PERRY                  What should wake Katy up tomorrow morning?    2  
Kim Kardashian West  RT @kkwbeauty: BOGO! Buy one Crème Contour Sti...    2  
Neil deGrasse Tyson  In 5-billion yrs the Sun will expand &amp; eng...    3  
Sebastian Ruder      New blog post giving an overview of softmax ap...    1

In [8]:
from sklearn.preprocessing import LabelEncoder

# Assuming 'author' is the column representing the classes
label_encoder = LabelEncoder()
df['Name'] = label_encoder.fit_transform(df['Name'])

In [9]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df['Tweet description'],df['Name'], stratify=df['Name'])

In [10]:
df['Name'].describe()

count    18680.000000
mean         3.504229
std          2.322235
min          0.000000
25%          2.000000
50%          4.000000
75%          6.000000
max          7.000000
Name: Name, dtype: float64

In [11]:
# Bert layers
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessed_text = bert_preprocess_model(text_input)
outputs = bert_model(preprocessed_text)

# Neural network layers
l = tf.keras.layers.Dropout(0.1, name="dropout")(outputs['pooled_output'])
l = tf.keras.layers.Dense(8, activation='softmax', name="output")(l)

# Use inputs and outputs to construct a final model
model = tf.keras.Model(inputs=[text_input], outputs = [l])

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


In [19]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 text (InputLayer)              [(None,)]            0           []                               
                                                                                                  
 keras_layer (KerasLayer)       {'input_type_ids':   0           ['text[0][0]']                   
                                (None, 128),                                                      
                                 'input_mask': (Non                                               
                                e, 128),                                                          
                                 'input_word_ids':                                                
                                (None, 128)}                                                  

In [22]:
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
model.fit(X_train, y_train, epochs=10)

Epoch 1/10
 73/438 [====>.........................] - ETA: 1:24:13 - loss: 2.1873 - accuracy: 0.1635